# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [89]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 
from pyspark.sql.functions import udf,desc,asc, isnan, count, when, col, sort_array,avg
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

spark = SparkSession \
        .builder \
        .appName("Wrangling Data") \
        .getOrCreate()

path = "data/sparkify_log_small.json"
df = spark.read.json(path)
df.show(n=2)

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|   Lily Allen|Logged In|Elizabeth|     F|            7|   Chase|195.23873| free|Shreveport-Bossie...|   PUT|NextSong|1512718541284|     5027|      

# Question 1

Which page did user id "" (empty string) NOT visit?

In [23]:
# TODO: write your code to answer question 1
blank_page = df.filter(df.userId == "").select(col('page')).distinct().collect()
blank_page

[Row(page='Home'), Row(page='About'), Row(page='Login'), Row(page='Help')]

In [25]:
all_page = df.select(col('page')).distinct().collect()
all_page

[Row(page='Submit Downgrade'),
 Row(page='Home'),
 Row(page='Downgrade'),
 Row(page='Logout'),
 Row(page='Save Settings'),
 Row(page='About'),
 Row(page='Settings'),
 Row(page='Login'),
 Row(page='NextSong'),
 Row(page='Help'),
 Row(page='Upgrade'),
 Row(page='Error'),
 Row(page='Submit Upgrade')]

In [30]:
answer1 = set(all_page) - set(blank_page)
answer1

{Row(page='Downgrade'),
 Row(page='Error'),
 Row(page='Logout'),
 Row(page='NextSong'),
 Row(page='Save Settings'),
 Row(page='Settings'),
 Row(page='Submit Downgrade'),
 Row(page='Submit Upgrade'),
 Row(page='Upgrade')}

In [32]:
for row in answer1:
    print(row.page)

Save Settings
Logout
Upgrade
Error
NextSong
Submit Upgrade
Submit Downgrade
Downgrade
Settings


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [45]:
# TODO: use this space to explore the behavior of the user with an empty string
q2 = df.filter(df.userId == "")

In [46]:
q2.select(col('auth')).dropDuplicates().show()

+----------+
|      auth|
+----------+
|Logged Out|
|     Guest|
+----------+



In [48]:
q2.select(col('page')).dropDuplicates().show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 3

How many female users do we have in the data set?

In [56]:
# TODO: write your code to answer question 3
df.filter(df.gender == 'F').select('userId').dropDuplicates().count()

462

# Question 4

How many songs were played from the most played artist?

In [68]:
# TODO: write your code to answer question 4
df.filter(df.artist.isNotNull()).groupby(df.artist).count().sort(col('count')).show()

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|        Mama's Jasje|    1|
|Silkk The Shocker...|    1|
|     Groove Coverage|    1|
|           Gold City|    1|
|           Catamenia|    1|
|              DJ Taz|    1|
|        STRATOVARIUS|    1|
|      The Chameleons|    1|
|      The Casualties|    1|
|      Nanci Griffith|    1|
|              Nebula|    1|
|Yonder Mountain S...|    1|
|        Generation X|    1|
|    Jane's Addiction|    1|
|        Ziggy Marley|    1|
|               Rufio|    1|
|  The Watts Prophets|    1|
|         Silverstein|    1|
|            Goldfish|    1|
|        Rhett Miller|    1|
+--------------------+-----+
only showing top 20 rows



In [69]:
df.filter(df.artist.isNotNull()).groupby(df.artist).count().orderBy(desc('count')).show()

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
|       Justin Bieber|   43|
|      The Black Keys|   40|
|         OneRepublic|   37|
|                Muse|   36|
|        Jack Johnson|   36|
|           Radiohead|   31|
|        Taylor Swift|   29|
|Barry Tuckwell/Ac...|   28|
|          Lily Allen|   28|
|               Train|   28|
|           Daft Punk|   27|
|           Metallica|   27|
|          Nickelback|   27|
|          Kanye West|   26|
|Red Hot Chili Pep...|   24|
+--------------------+-----+
only showing top 20 rows



In [81]:
df.filter(df.artist.isNotNull()) \
    .groupBy('Artist')\
    .agg({'Artist':'count'}) \
    .withColumnRenamed('count(Artist)','count')\
    .sort(desc('count')).show()

+--------------------+-----+
|              Artist|count|
+--------------------+-----+
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
|       Justin Bieber|   43|
|      The Black Keys|   40|
|         OneRepublic|   37|
|                Muse|   36|
|        Jack Johnson|   36|
|           Radiohead|   31|
|        Taylor Swift|   29|
|Barry Tuckwell/Ac...|   28|
|          Lily Allen|   28|
|               Train|   28|
|           Daft Punk|   27|
|           Metallica|   27|
|          Nickelback|   27|
|          Kanye West|   26|
|Red Hot Chili Pep...|   24|
+--------------------+-----+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [90]:
# TODO: write your code to answer question 5
filter_func = udf(lambda x : int(x == 'Home'), IntegerType())
window = Window.partitionBy('userID')\
                    .orderBy(desc('ts'))\
                    .rangeBetween(Window.unboundedPreceding, 0)

In [92]:
cumsum = df.filter((df.page == 'NextSong') | (df.page == 'Home')) \
            .select('userID','page','ts').show(n=5)

+------+--------+-------------+
|userID|    page|           ts|
+------+--------+-------------+
|  1046|NextSong|1513720872284|
|  1000|NextSong|1513720878284|
|  2219|NextSong|1513720881284|
|  2373|NextSong|1513720905284|
|  1747|    Home|1513720913284|
+------+--------+-------------+
only showing top 5 rows



In [93]:
cumsum = df.filter((df.page == 'NextSong') | (df.page == 'Home')) \
            .select('userID','page','ts') \
            .withColumn('homevisit', filter_func(df.page)).show(n=5)

+------+--------+-------------+---------+
|userID|    page|           ts|homevisit|
+------+--------+-------------+---------+
|  1046|NextSong|1513720872284|        0|
|  1000|NextSong|1513720878284|        0|
|  2219|NextSong|1513720881284|        0|
|  2373|NextSong|1513720905284|        0|
|  1747|    Home|1513720913284|        1|
+------+--------+-------------+---------+
only showing top 5 rows



In [ ]:
cumsum = df.filter((df.page == 'NextSong') | (df.page == 'Home')) \
            .select('userID','page','ts') \
            .withColumn('homevisit', filter_func(df.page))\
            .withColumn('period', Fsum(col('homevisit')).over(window)).show(n=20)

In [106]:
cumsum = df.filter((df.page == 'NextSong') | (df.page == 'Home')) \
            .select('userID','page','ts') \
            .withColumn('homevisit', filter_func(df.page))\
            .withColumn('period', Fsum(col('homevisit')).over(window))

In [102]:
cumsum.filter((cumsum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}).show(n=5)


+------+------+-------------+
|userID|period|count(period)|
+------+------+-------------+
|  1436|     0|            2|
|  2088|     1|           13|
|  2162|     0|           19|
|  2162|     2|           15|
|  2294|     0|            4|
+------+------+-------------+
only showing top 5 rows



In [107]:
cumsum.filter((cumsum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

